In [0]:
pip install yfinance


Python interpreter will be restarted.
Collecting yfinance
 Downloading yfinance-0.1.55.tar.gz (23 kB)
Requirement already satisfied: pandas>=0.24 in /databricks/python3/lib/python3.7/site-packages (from yfinance) (1.0.1)
Requirement already satisfied: numpy>=1.15 in /databricks/python3/lib/python3.7/site-packages (from yfinance) (1.18.1)
Requirement already satisfied: requests>=2.20 in /databricks/python3/lib/python3.7/site-packages (from yfinance) (2.22.0)
Collecting lxml>=4.5.1
 Downloading lxml-4.6.2-cp37-cp37m-manylinux1_x86_64.whl (5.5 MB)
Collecting multitasking>=0.0.7
 Downloading multitasking-0.0.9.tar.gz (8.1 kB)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.24->yfinance) (2.8.1)
Requirement already satisfied: numpy>=1.15 in /databricks/python3/lib/python3.7/site-packages (from yfinance) (1.18.1)
Requirement already satisfied: pytz>=2017.2 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.24->yfinance) (2019.3)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.7/site-packages (from python-dateutil>=2.6.1->pandas>=0.24->yfinance) (1.14.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.20->yfinance) (2020.6.20)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.20->yfinance) (2.8)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.20->yfinance) (1.25.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests>=2.20->yfinance) (3.0.4)
Building wheels for collected packages: yfinance, multitasking
 Building wheel for yfinance (setup.py): started
 Building wheel for yfinance (setup.py): finished with status 'done'
 Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=ac5d5aaca4486d444fd74a57825198cb3cfebec376acd7cf78e04951a15445a2
 Stored in directory: /root/.cache/pip/wheels/aa/8a/36/59ed4f6fbcb6100967618eeb0696046bf9777a41ac2ff1f9b9
 Building wheel for multitasking (setup.py): started
 Building wheel for multitasking (setup.py): finished with status 'done'
 Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8366 sha256=6e78e0268df192a14ceba2855ee21fd73923346b298eface53e23bc4b778cf98
 Stored in directory: /root/.cache/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
Installing collected packages: multitasking, lxml, yfinance
Successfully installed lxml-4.6.2 multitasking-0.0.9 yfinance-0.1.55
Python interpreter will be restarted.

## importing libraries

In [0]:
from pyspark.sql import SparkSession

import numpy as np
import requests
import csv
import matplotlib.pyplot as plt
import warnings
from matplotlib import dates
import matplotlib as mlp
%matplotlib inline
from pyspark.sql.types import StructType,StructField, StringType
from pyspark import SparkContext
from pyspark.sql import SQLContext 
sql = SQLContext(sc)
import yfinance as yf


from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer,VectorIndexer,OneHotEncoder,VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StandardScaler

from pyspark.mllib.stat import Statistics
import pandas as pd

### Reading data

In [0]:
#loading the consituents file[symbols, industry, sector] for S&P 500 companies:
ticker_names = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/baradi@usf.edu/constituents_csv.csv",inferSchema=True,header=True)

#ticker symbols in list
ticker_name = ticker_names.select('Symbol').collect()
ticker_name[:2]

Out[2]: [Row(Symbol='MMM'), Row(Symbol='AOS')]

### Variables considered:

In [0]:
# final file reading(all 3 files merged - info, recommendations, sustainabilit)

df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/baradi@usf.edu/stocks_final_data-1.csv",inferSchema=True,header=True)

In [0]:

# Variable selection

df = df1.select(
 
  #company details
  
  'ticker',
 'Sector',
 'longBusinessSummary',
 'fullTimeEmployees',
 'Industry',
 'shortName',
  
 #sustainability
 'environmentScore',
 'governanceScore',
 'socialScore',
  
  # Regular market details

 'regularMarketOpen',
 'twoHundredDayAverage',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
 'Currency',
 
  # Day share details

 'previousClose',
 'volume24Hr',
 'forwardPE',
 'averageVolume',
 'averageDailyVolume10Day',
 
  
 # >52week details

 'fiftyTwoWeekHigh',
 'fiftyTwoWeekLow',
 '52WeekChange',
 'SandP52WeekChange',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'payoutRatio',
 'earningsQuarterlyGrowth',
 
  #yearly details

 'fiveYearAverageReturn',
 'fiveYearAvgDividendYield',
 'beta3Year',
 'threeYearAverageReturn',
 
  # Company growth details

 'dividendRate',
 'marketCap',
 'beta',
 'pegRatio',
 'lastDividendValue',
 'Volume',
 'yield',
 'dividendYield',
 'enterpriseToRevenue',
 'enterpriseToEbitda',
 'forwardEps',
 'revenueQuarterlyGrowth',
 'sharesOutstanding',
 'bookValue',
 'heldPercentInsiders',
 'floatShares',
 'earningsQuarterlyGrowth',
 'shortPercentOfFloat',
 'sharesShortPriorMonth',
 'profitMargins',
 'trailingAnnualDividendYield',
 'trailingAnnualDividendRate',
 'payoutRatio',
 'dividendRate',
 'trailingPE',
 'forwardPE',
 'trailingEps',
 'shortRatio',
  
  # recommendation ratings
  
 'Goldman Sachs',
 'JP Morgan',
 'Morgan Stanley',
# 'CItigroup',
 'Barclays',
 'UBS',
 'Bank of America',
# 'Deutsche Bank',
 'Credit Suisse',
# 'Wells Fargo'
)
df.createOrReplaceTempView("stocks_data_table")
df.show(2)


+------+-----------+--------------------+-----------------+--------------------+--------------------+----------------+---------------+-----------+-----------------+--------------------+--------------------+-------------------+--------------------------+-------------------+--------+-------------+----------+------------------+-------------+-----------------------+----------------+---------------+-------------------+-----------------+---------------+--------------------+-----------+-----------------------+---------------------+------------------------+---------+----------------------+------------+-------------+------------------+--------+-----------------+-----------+-----+--------------------+-------------------+------------------+----------+----------------------+-----------------+-------------------+--------------------+------------+-----------------------+-------------------+---------------------+--------------------+---------------------------+--------------------------+-----------+------------+----------+------------------+-----------+----------+-------------+---------+--------------+--------+----+---------------+-------------+
ticker| Sector| longBusinessSummary|fullTimeEmployees| Industry| shortName|environmentScore|governanceScore|socialScore|regularMarketOpen|twoHundredDayAverage|regularMarketDayHigh|regularMarketDayLow|regularMarketPreviousClose|regularMarketVolume|Currency|previousClose|volume24Hr| forwardPE|averageVolume|averageDailyVolume10Day|fiftyTwoWeekHigh|fiftyTwoWeekLow| 52WeekChange|SandP52WeekChange|fiftyDayAverage|twoHundredDayAverage|payoutRatio|earningsQuarterlyGrowth|fiveYearAverageReturn|fiveYearAvgDividendYield|beta3Year|threeYearAverageReturn|dividendRate| marketCap| beta|pegRatio|lastDividendValue| Volume|yield| dividendYield|enterpriseToRevenue|enterpriseToEbitda|forwardEps|revenueQuarterlyGrowth|sharesOutstanding| bookValue| heldPercentInsiders| floatShares|earningsQuarterlyGrowth|shortPercentOfFloat|sharesShortPriorMonth| profitMargins|trailingAnnualDividendYield|trailingAnnualDividendRate|payoutRatio|dividendRate|trailingPE| forwardPE|trailingEps|shortRatio|Goldman Sachs|JP Morgan|Morgan Stanley|Barclays| UBS|Bank of America|Credit Suisse|
+------+-----------+--------------------+-----------------+--------------------+--------------------+----------------+---------------+-----------+-----------------+--------------------+--------------------+-------------------+--------------------------+-------------------+--------+-------------+----------+------------------+-------------+-----------------------+----------------+---------------+-------------------+-----------------+---------------+--------------------+-----------+-----------------------+---------------------+------------------------+---------+----------------------+------------+-------------+------------------+--------+-----------------+-----------+-----+--------------------+-------------------+------------------+----------+----------------------+-----------------+-------------------+--------------------+------------+-----------------------+-------------------+---------------------+--------------------+---------------------------+--------------------------+-----------+------------+----------+------------------+-----------+----------+-------------+---------+--------------+--------+----+---------------+-------------+
 A| Healthcare|Agilent Technolog...| 16300.0|Diagnostics & Res...|Agilent Technolog...| 0.87| 6.45| 9.4| 108.64| 96.17864| 112.3| 108.315| 108.68| 2700586| USD| 108.68| null|30.133152000000003| 1260625| 1769771| 117.5| 61.13|0.38163459999999993| 0.13527405| 106.65143| 96.17864| 0.3171| 0.042| null| 0.88| null| null| 0.78|34188496896.0| 1.032694| 3.58| 0.18| 2700586.0| null|0.006999999999999999| 6.766| 26.772| 3.68| null| 308310016.0| 16.142| 0.00381|3.07218219E8| 0.042| 0.010199999| 2620892.0| 0.13229999| 0.006477733| 0.7040000000000001| 0.3171| 0.78| 50.063206|30.133152000000003| 2.215| 3.08| main| main| main| main|main| down| init|


### Null values imputation

In [0]:
%sql
select * from stocks_data_table 

ticker,Sector,longBusinessSummary,fullTimeEmployees,Industry,shortName,environmentScore,governanceScore,socialScore,regularMarketOpen,twoHundredDayAverage,regularMarketDayHigh,regularMarketDayLow,regularMarketPreviousClose,regularMarketVolume,Currency,previousClose,volume24Hr,forwardPE,averageVolume,averageDailyVolume10Day,fiftyTwoWeekHigh,fiftyTwoWeekLow,52WeekChange,SandP52WeekChange,fiftyDayAverage,twoHundredDayAverage,payoutRatio,earningsQuarterlyGrowth,fiveYearAverageReturn,fiveYearAvgDividendYield,beta3Year,threeYearAverageReturn,dividendRate,marketCap,beta,pegRatio,lastDividendValue,Volume,yield,dividendYield,enterpriseToRevenue,enterpriseToEbitda,forwardEps,revenueQuarterlyGrowth,sharesOutstanding,bookValue,heldPercentInsiders,floatShares,earningsQuarterlyGrowth,shortPercentOfFloat,sharesShortPriorMonth,profitMargins,trailingAnnualDividendYield,trailingAnnualDividendRate,payoutRatio,dividendRate,trailingPE,forwardPE,trailingEps,shortRatio,Goldman Sachs,JP Morgan,Morgan Stanley,Barclays,UBS,Bank of America,Credit Suisse
A,Healthcare,"Agilent Technologies, Inc. provides application focused solutions to the life sciences, diagnostics, and applied chemical markets worldwide. It operates in three segments: Life Sciences and Applied Markets, Diagnostics and Genomics, and Agilent CrossLab. The Life Sciences and Applied Markets segment offers liquid and gas chromatography systems and components; liquid and gas chromatography mass spectrometry systems; inductively coupled plasma mass spectrometry instruments; atomic absorption instruments; microwave plasma-atomic emission spectrometry instruments; inductively coupled plasma optical emission spectrometry instruments; raman spectroscopy; cell analysis plate based assays; flow cytometer; real-time cell analyzer; cell imaging systems; microplate readers; laboratory software and information management and analytics; laboratory automation and robotic systems; dissolution testing; vacuum pumps; and measurement technologies. The Diagnostics and Genomics segment provides reagents, instruments, software, and consumables; arrays for DNA mutation detection, genotyping, gene copy number determination, identification of gene rearrangements, DNA methylation profiling, and gene expression profiling, as well as sequencing target enrichment, genetic data management, and interpretation support software; and equipment focused on production of synthesized oligonucleotides for use as active pharmaceutical ingredients. The Agilent CrossLab segment offers GC and LC columns, sample preparation products, custom chemistries, and laboratory instrument supplies; and startup, operational, training, compliance support, and software as a service, as well as asset management and consultation services. The company markets its products through direct sales, distributors, resellers, manufacturer's representatives, and electronic commerce. It has collaboration agreement with SGS AXYS. The company was founded in 1999 and is headquartered in Santa Clara, California.",16300.0,Diagnostics & Research,"Agilent Technologies, Inc.",0.87,6.45,9.4,108.64,96.17864,112.3,108.315,108.68,2700586,USD,108.68,null,30.133152000000003,1260625,1769771,117.5,61.13,0.38163459999999993,0.13527405,106.65143,96.17864,0.3171,0.042,null,0.88,null,null,0.78,34188496896.0,1.032694,3.58,0.18,2700586.0,null,0.006999999999999999,6.766,26.772,3.68,null,308310016.0,16.142,0.00381,3.07218219E8,0.042,0.010199999,2620892.0,0.13229999,0.006477733,0.7040000000000001,0.3171,0.78,50.063206,30.133152000000003,2.215,3.08,main,main,main,main,main,down,init
AAL,Industrials,"American Airlines Group Inc., through its subsidiaries, operates as a network air carrier. It provides scheduled air transportation services for passengers and cargo. As of December 31, 2019, the company operated a mainline fleet of 942 aircraft. It serves 365 destinations in approximately 61 countries, principally from its hubs and gateways in Charlotte, Chicago, Dallas/Fort Worth, Londo

In [0]:
#checking the count of null values

from pyspark.sql.functions import isnan, when, count, col

df_null_count = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
df_null_count.createOrReplaceTempView("null_count_table")


In [0]:
%sql
select * from null_count_table

ticker,Sector,longBusinessSummary,fullTimeEmployees,Industry,shortName,environmentScore,governanceScore,socialScore,regularMarketOpen,twoHundredDayAverage,regularMarketDayHigh,regularMarketDayLow,regularMarketPreviousClose,regularMarketVolume,Currency,previousClose,volume24Hr,forwardPE,averageVolume,averageDailyVolume10Day,fiftyTwoWeekHigh,fiftyTwoWeekLow,52WeekChange,SandP52WeekChange,fiftyDayAverage,twoHundredDayAverage,payoutRatio,earningsQuarterlyGrowth,fiveYearAverageReturn,fiveYearAvgDividendYield,beta3Year,threeYearAverageReturn,dividendRate,marketCap,beta,pegRatio,lastDividendValue,Volume,yield,dividendYield,enterpriseToRevenue,enterpriseToEbitda,forwardEps,revenueQuarterlyGrowth,sharesOutstanding,bookValue,heldPercentInsiders,floatShares,earningsQuarterlyGrowth,shortPercentOfFloat,sharesShortPriorMonth,profitMargins,trailingAnnualDividendYield,trailingAnnualDividendRate,payoutRatio,dividendRate,trailingPE,forwardPE,trailingEps,shortRatio,Goldman Sachs,JP Morgan,Morgan Stanley,Barclays,UBS,Bank of America,Credit Suisse
0,2,2,7,2,0,1,1,1,0,0,0,0,0,0,0,1,451,5,0,0,0,0,376,376,0,0,64,78,452,89,452,451,104,0,2,3,387,0,450,104,2,30,6,450,2,6,2,2,78,4,3,2,76,76,64,104,74,5,7,3,36,35,47,48,49,50,68


In [0]:
#dropping columns with null_values>300
df  = df.drop('volume24hr',
 '52WeekChange',
 'SandP52WeekChange',
 'fiveYearAverageReturn',
 'beta3Year',
 'threeYearAverageReturn',
 'lastDividendValue',
 'yield',
 'revenueQuarterlyGrowth')

#dropping rows with null values <10
df = df.na.drop(subset=["Sector",'longBusinessSummary','Industry','previousClose','governanceScore','previousClose','pegRatio','shortPercentOfFloat'])


# replacing null values with mean grouped by industry
null_columns = ['payoutRatio',
                   'earningsQuarterlyGrowth',
                   'fiveYearAvgDividendYield',
                   'earningsQuarterlyGrowth',
                   'trailingAnnualDividendRate',
                   'trailingPE',
                   'dividendRate',
                   'dividendYield',
                   'socialScore',
                   'governanceScore',
                   'environmentScore',
                   'fullTimeEmployees',
                   'previousClose',
                   'forwardPE',
                   'beta',
                   'pegRatio',
                   'enterpriseToRevenue',
                   'sharesOutstanding',
                   'bookValue',
                   'heldPercentInsiders',
                   'floatShares',
                   'shortPercentOfFloat',
                   'sharesShortPriorMonth',
                   'profitMargins',
                   'trailingEps',
                   'shortRatio',
                    'enterpriseToEbitda',
                    'forwardEps'
                 ]
  
  
# imputing null values of the numerical columns with partition over industry

from pyspark.sql import Window
from pyspark.sql.functions import *

category_window = Window.partitionBy("Industry")
value_mean = mean("value0").over(category_window)

for i in null_columns:
   
  category_window = Window.partitionBy("Industry")
  value_mean = mean( "value0").over(category_window)
 
  if i== 'payoutRatio':
      df_new =df.withColumn("value0", coalesce(i, lit(0)))\
      .withColumn("value_mean", value_mean )\
      .withColumn(i+ '_new', coalesce(i, value_mean))
  else:
      df_new  = df_new.withColumn("value0", coalesce(i, lit(0)))\
      .withColumn("value_mean", value_mean )\
      .withColumn(i+'_new', coalesce(i, value_mean))


In [0]:
# checking again the null values
null_count_replaced = df_new.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_new.columns])
null_count_replaced.createOrReplaceTempView("not_null_table")


In [0]:
%sql
select * from not_null_table

ticker,Sector,longBusinessSummary,fullTimeEmployees,Industry,shortName,environmentScore,governanceScore,socialScore,regularMarketOpen,twoHundredDayAverage,regularMarketDayHigh,regularMarketDayLow,regularMarketPreviousClose,regularMarketVolume,Currency,previousClose,forwardPE,averageVolume,averageDailyVolume10Day,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiftyDayAverage,twoHundredDayAverage,payoutRatio,earningsQuarterlyGrowth,fiveYearAvgDividendYield,dividendRate,marketCap,beta,pegRatio,Volume,dividendYield,enterpriseToRevenue,enterpriseToEbitda,forwardEps,sharesOutstanding,bookValue,heldPercentInsiders,floatShares,earningsQuarterlyGrowth,shortPercentOfFloat,sharesShortPriorMonth,profitMargins,trailingAnnualDividendYield,trailingAnnualDividendRate,payoutRatio,dividendRate,trailingPE,forwardPE,trailingEps,shortRatio,Goldman Sachs,JP Morgan,Morgan Stanley,Barclays,UBS,Bank of America,Credit Suisse,value0,value_mean,payoutRatio_new,earningsQuarterlyGrowth_new,fiveYearAvgDividendYield_new,trailingAnnualDividendRate_new,trailingPE_new,dividendRate_new,dividendYield_new,socialScore_new,governanceScore_new,environmentScore_new,fullTimeEmployees_new,previousClose_new,forwardPE_new,beta_new,pegRatio_new,enterpriseToRevenue_new,sharesOutstanding_new,bookValue_new,heldPercentInsiders_new,floatShares_new,shortPercentOfFloat_new,sharesShortPriorMonth_new,profitMargins_new,trailingEps_new,shortRatio_new,enterpriseToEbitda_new,forwardEps_new
0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,61,75,87,102,0,0,0,0,102,0,28,4,0,4,0,0,75,0,0,0,74,74,61,102,73,5,4,0,33,33,43,44,46,47,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Creating dataframe with final variables

df_final = df_new.select(
'ticker',
'Sector',
 #'longBusinessSummary',
 'Industry',
# 'shortName',
 'regularMarketOpen',
 'twoHundredDayAverage',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
# 'Currency',
 'averageVolume',
 'averageDailyVolume10Day',
 'fiftyTwoWeekHigh',
 'fiftyTwoWeekLow',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'marketCap',
 'Volume',
# 'forwardEps',
#  'revenueQuarterlyGrowth',
#  'trailingAnnualDividendYield',
#  'Goldman Sachs',
 'payoutRatio_new',
 'earningsQuarterlyGrowth_new',
 'fiveYearAvgDividendYield_new',
 'trailingAnnualDividendRate_new',
 'trailingPE_new',
 'dividendRate_new',
 'dividendYield_new',
 'socialScore_new',
 'governanceScore_new',
 'environmentScore_new',
 'fullTimeEmployees_new',
 'previousClose_new',
 'beta_new',
 'pegRatio_new',
 'enterpriseToRevenue_new',
 'sharesOutstanding_new',
 'bookValue_new',
 'heldPercentInsiders_new',
 'floatShares_new',
 'shortPercentOfFloat_new',
 'sharesShortPriorMonth_new',
 'profitMargins_new',
 'trailingEps_new',
 'shortRatio_new',
  
 # 'fullTimeEmployees',
  'forwardPE_new',
  'previousClose',
  'enterpriseToEbitda_new',

 )

## Exploratory Data Analysis

In [0]:
print('number of columns :' + str(len(df_final.columns)))
print('number of tickers taken :' + str( df_final.count()))


number of columns :44
number of tickers taken :446

In [0]:
df_final.describe().show()

+-------+------+---------------+--------------------+------------------+--------------------+--------------------+-------------------+--------------------------+-------------------+--------------------+-----------------------+------------------+------------------+------------------+--------------------+--------------------+-----------------+------------------+---------------------------+----------------------------+------------------------------+-----------------+------------------+--------------------+------------------+-------------------+--------------------+---------------------+------------------+------------------+-----------------+-----------------------+---------------------+-----------------+-----------------------+--------------------+-----------------------+-------------------------+-------------------+-------------------+-----------------+-----------------+------------------+----------------------+
summary|ticker| Sector| Industry| regularMarketOpen|twoHundredDayAverage|regularMarketDayHigh|regularMarketDayLow|regularMarketPreviousClose|regularMarketVolume| averageVolume|averageDailyVolume10Day| fiftyTwoWeekHigh| fiftyTwoWeekLow| fiftyDayAverage|twoHundredDayAverage| marketCap| Volume| payoutRatio_new|earningsQuarterlyGrowth_new|fiveYearAvgDividendYield_new|trailingAnnualDividendRate_new| trailingPE_new| dividendRate_new| dividendYield_new| socialScore_new|governanceScore_new|environmentScore_new|fullTimeEmployees_new| previousClose_new| beta_new| pegRatio_new|enterpriseToRevenue_new|sharesOutstanding_new| bookValue_new|heldPercentInsiders_new| floatShares_new|shortPercentOfFloat_new|sharesShortPriorMonth_new| profitMargins_new| trailingEps_new| shortRatio_new| forwardPE_new| previousClose|enterpriseToEbitda_new|
+-------+------+---------------+--------------------+------------------+--------------------+--------------------+-------------------+--------------------------+-------------------+--------------------+-----------------------+------------------+------------------+------------------+--------------------+--------------------+-----------------+------------------+---------------------------+----------------------------+------------------------------+-----------------+------------------+--------------------+------------------+-------------------+--------------------+---------------------+------------------+------------------+-----------------+-----------------------+---------------------+-----------------+-----------------------+--------------------+-----------------------+-------------------------+-------------------+-------------------+-----------------+-----------------+------------------+----------------------+
 count| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446| 446|
 mean| null| null| null| 159.7821502242154| 144.96526730784754| 161.033092825112| 157.82189125560555| 159.80831838565038| 4543406.569506726| 5228424.226457399| 5106438.70852018|180.37547692825106| 87.84836891928255|154.65080638565024| 144.96526730784754|6.586666229409865...|4543406.569506726|0.6317301075853333| 0.3292642521134017| 2.1353805786888747| 1.8817886842479876|56.53315536429353|1.8765681187273127|0.020848732388917362|10.175560538116592| 7.2704708520179455| 5.911255605381165| 54917.47477578474|159.80831838565038|1.0949575403587433| 3.35358744394619| 4.9788856502242185| 6.460766620941705E8| 34.2547060948445| 0.034360112345291474| 6.252994878766816E8| 0.033195291479820616| 1.1532742378923766E7|0.10398970877130044| 4.737628784841291|3.154237668161437|35.15471090992262|159.80831838565038| 15.021861266816142|
 stddev| null| null| null|305.23820619214723| 276.9332288700516| 306.4927339997984| 300.76271369974063| 304.5128623101729| 8625676.825730084|1.1593910719436005E7| 1.0620125118870482E7| 330.7070138002311|156.05169175226874|301.46596481269927| 276.9332288700516|1.6455672

In [0]:
df_final.printSchema()

root
-- ticker: string (nullable = true)
-- Sector: string (nullable = true)
-- Industry: string (nullable = true)
-- regularMarketOpen: string (nullable = true)
-- twoHundredDayAverage: double (nullable = true)
-- regularMarketDayHigh: string (nullable = true)
-- regularMarketDayLow: string (nullable = true)
-- regularMarketPreviousClose: string (nullable = true)
-- regularMarketVolume: string (nullable = true)
-- averageVolume: integer (nullable = true)
-- averageDailyVolume10Day: integer (nullable = true)
-- fiftyTwoWeekHigh: double (nullable = true)
-- fiftyTwoWeekLow: double (nullable = true)
-- fiftyDayAverage: double (nullable = true)
-- twoHundredDayAverage: double (nullable = true)
-- marketCap: string (nullable = true)
-- Volume: double (nullable = true)
-- payoutRatio_new: string (nullable = true)
-- earningsQuarterlyGrowth_new: double (nullable = true)
-- fiveYearAvgDividendYield_new: double (nullable = true)
-- trailingAnnualDividendRate_new: string (nullable = true)
-- trailingPE_new: string (nullable = true)
-- dividendRate_new: double (nullable = true)
-- dividendYield_new: double (nullable = true)
-- socialScore_new: double (nullable = true)
-- governanceScore_new: string (nullable = true)
-- environmentScore_new: string (nullable = true)
-- fullTimeEmployees_new: double (nullable = true)
-- previousClose_new: string (nullable = true)
-- beta_new: double (nullable = true)
-- pegRatio_new: string (nullable = true)
-- enterpriseToRevenue_new: double (nullable = true)
-- sharesOutstanding_new: string (nullable = true)
-- bookValue_new: double (nullable = true)
-- heldPercentInsiders_new: double (nullable = true)
-- floatShares_new: double (nullable = true)
-- shortPercentOfFloat_new: double (nullable = true)
-- sharesShortPriorMonth_new: double (nullable = true)
-- profitMargins_new: string (nullable = true)
-- trailingEps_new: double (nullable = true)
-- shortRatio_new: double (nullable = true)
-- forwardPE_new: double (nullable = true)
-- previousClose: string (nullable = true)
-- enterpriseToEbitda_new: double (nullable = true)

In [0]:
df_final.groupby('sector').count().show()

+--------------------+-----+
 sector|count|
+--------------------+-----+
 Consumer Defensive| 33|
 Financial Services| 63|
Communication Ser...| 17|
 Energy| 22|
 Healthcare| 57|
 Real Estate| 28|
 Utilities| 27|
 Industrials| 61|
 Technology| 60|
 Consumer Cyclical| 60|
 Basic Materials| 18|
+--------------------+-----+

In [0]:
df_final.groupby('sector','industry').count().show()

+------------------+--------------------+-----+
 sector| industry|count|
+------------------+--------------------+-----+
Consumer Defensive| Packaged Foods| 7|
Consumer Defensive| Confectioners| 2|
 Consumer Cyclical| Specialty Retail| 7|
 Consumer Cyclical| Leisure| 1|
Consumer Defensive|Beverages—Winerie...| 1|
Financial Services|Insurance—Diversi...| 3|
 Healthcare|Medical Instrumen...| 9|
 Utilities|Utilities—Diversi...| 7|
Financial Services| Banks—Regional| 12|
 Utilities|Utilities—Regulat...| 3|
 Consumer Cyclical|Home Improvement ...| 2|
 Consumer Cyclical|Packaging & Conta...| 4|
 Consumer Cyclical| Personal Services| 1|
 Industrials|Industrial Distri...| 1|
 Basic Materials| Steel| 1|
 Basic Materials| Agricultural Inputs| 3|
 Consumer Cyclical| Restaurants| 6|
Consumer Defensive| Beverages—Brewers| 1|
 Consumer Cyclical| Luxury Goods| 2|
 Consumer Cyclical| Department Stores| 1|
+------------------+--------------------+-----+
only showing top 20 rows

In [0]:
df_final.columns


Out[16]: ['ticker',
 'Sector',
 'Industry',
 'regularMarketOpen',
 'twoHundredDayAverage',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
 'averageVolume',
 'averageDailyVolume10Day',
 'fiftyTwoWeekHigh',
 'fiftyTwoWeekLow',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'marketCap',
 'Volume',
 'payoutRatio_new',
 'earningsQuarterlyGrowth_new',
 'fiveYearAvgDividendYield_new',
 'trailingAnnualDividendRate_new',
 'trailingPE_new',
 'dividendRate_new',
 'dividendYield_new',
 'socialScore_new',
 'governanceScore_new',
 'environmentScore_new',
 'fullTimeEmployees_new',
 'previousClose_new',
 'beta_new',
 'pegRatio_new',
 'enterpriseToRevenue_new',
 'sharesOutstanding_new',
 'bookValue_new',
 'heldPercentInsiders_new',
 'floatShares_new',
 'shortPercentOfFloat_new',
 'sharesShortPriorMonth_new',
 'profitMargins_new',
 'trailingEps_new',
 'shortRatio_new',
 'forwardPE_new',
 'previousClose',
 'enterpriseToEbitda_new']

## correlation matrix

In [0]:

num_cols = ['twoHundredDayAverage',
 'averageVolume',
 'averageDailyVolume10Day',
 'fiftyTwoWeekHigh',
 'fiftyTwoWeekLow',
 'fiftyDayAverage',
# 'twoHundredDayAverage',
 'Volume',
#  'forwardEps',
 'earningsQuarterlyGrowth_new',
 'fiveYearAvgDividendYield_new',
 'dividendRate_new',
 'dividendYield_new',
 'socialScore_new']
#  'fullTimeEmployees_new',
#  'beta_new',
#  'enterpriseToRevenue_new',
#  'bookValue_new',
#  'heldPercentInsiders_new',
#  'floatShares_new',
#  'shortPercentOfFloat_new',
#  'sharesShortPriorMonth_new',
#  'trailingEps_new',
#  'shortRatio_new',
#  'fullTimeEmployees',
#  'forwardPE_new',
#  'enterpriseToEbitda_new']

corr_data = df_final.select(num_cols)

col_names = corr_data.columns
features = corr_data.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
#for better visibility we used pandas dataframe but correlation is calculated using the RDD
#corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names

to_drop = [column for column in corr_df.columns if any(corr_df[column] > 0.5)]

print('Variables with correlation greater than 0.7 : ',to_drop)
corr_df


# kot = corr_df[corr_df>=.9]


,twoHundredDayAverage,averageVolume,averageDailyVolume10Day,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiftyDayAverage,Volume,earningsQuarterlyGrowth_new,fiveYearAvgDividendYield_new,dividendRate_new,dividendYield_new,socialScore_new
twoHundredDayAverage,1.000000,-0.111557,-0.131331,0.998132,0.989941,0.998813,-0.136019,0.038438,-0.269647,0.070221,-0.216751,0.023926
averageVolume,-0.111557,1.000000,0.932322,-0.106190,-0.119939,-0.109962,0.931566,0.014947,0.157102,-0.156249,0.006860,0.129651
averageDailyVolume10Day,-0.131331,0.932322,1.000000,-0.123760,-0.139797,-0.130042,0.975895,0.012891,0.210780,-0.163958,0.017301,0.124532
fiftyTwoWeekHigh,0.998132,-0.106190,-0.123760,1.000000,0.987327,0.998107,-0.130951,0.036403,-0.262533,0.069349,-0.211624,0.023261
fiftyTwoWeekLow,0.989941,-0.119939,-0.139797,0.987327,1.000000,0.986087,-0.144081,0.033040,-0.267354,0.092792,-0.211396,0.024107
fiftyDayAverage,0.998813,-0.109962,-0.130042,0.998107,0.986087,1.000000,-0.134650,0.039290,-0.266961,0.066402,-0.217653,0.018117
Volume,-0.136019,0.931566,0.975895,-0.130951,-0.144081,-0.134650,1.000000,0.016614,0.209090,-0.171091,0.029671,0.138491
earningsQuarterlyGrowth_new,0.038438,0.014947,0.012891,0.036403,0.033040,0.039290,0.016614,1.000000,-0.085784,-0.024116,-0.124083,0.029715
fiveYearAvgDividendYield_new,-0.269647,0.157102,0.210780,-0.262533,-0.267354,-0.266961,0.209090,-0.085784,1.000000,0.275951,0.708834,-0.184881
dividendRate_new,0.070221,-0.156249,-0.163958,0.069349,0.092792,0.066402,-0.171091,-0.024116,0.275951,1.000000,0.377873,-0.062638


####  removing the columns with correlation greater than 0.9


'fiftyTwoWeekHigh',
'fiftyTwoWeekLow',
'twoHundredDayAverage',

In [0]:
# Creating dataframe with final variables

df_final = df_final.select(
'ticker',
 'Sector',
 #'longBusinessSummary',
 'Industry',
# 'shortName',
 'regularMarketOpen',
#  'twoHundredDayAverage',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
# 'Currency',
 'averageVolume',
 'averageDailyVolume10Day',
#  'fiftyTwoWeekHigh',
#  'fiftyTwoWeekLow',
 'fiftyDayAverage',
#  'twoHundredDayAverage',
 'marketCap',
 'Volume',
# 'forwardEps',
#  'revenueQuarterlyGrowth',
#  'trailingAnnualDividendYield',
#  'Goldman Sachs',
 'payoutRatio_new',
 'earningsQuarterlyGrowth_new',
 'fiveYearAvgDividendYield_new',
 'trailingAnnualDividendRate_new',
 'trailingPE_new',
 'dividendRate_new',
 'dividendYield_new',
 'socialScore_new',
 'governanceScore_new',
 'environmentScore_new',
 'fullTimeEmployees_new',
 'previousClose_new',
 'beta_new',
 'pegRatio_new',
 'enterpriseToRevenue_new',
 'sharesOutstanding_new',
 'bookValue_new',
 'heldPercentInsiders_new',
 'floatShares_new',
 'shortPercentOfFloat_new',
 'sharesShortPriorMonth_new',
 'profitMargins_new',
 'trailingEps_new',
 'shortRatio_new',
  
 # 'fullTimeEmployees',
  'forwardPE_new',
  'previousClose',
  'enterpriseToEbitda_new',

 )

In [0]:

# casting multiple columns(string) to double in df_final dataframe

casting_columns = ['regularMarketOpen',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
 'marketCap',
 'payoutRatio_new',
 'trailingAnnualDividendRate_new',
 'trailingPE_new',
 'governanceScore_new',
 'environmentScore_new',
 'previousClose_new',
 'pegRatio_new',
 'sharesOutstanding_new',
 'profitMargins_new',
 'previousClose']

from pyspark.sql.functions import col
for col_name in casting_columns:
    df_final= df_final.withColumn(col_name, col(col_name).cast('double'))
df_final.printSchema()

root
-- ticker: string (nullable = true)
-- Sector: string (nullable = true)
-- Industry: string (nullable = true)
-- regularMarketOpen: double (nullable = true)
-- regularMarketDayHigh: double (nullable = true)
-- regularMarketDayLow: double (nullable = true)
-- regularMarketPreviousClose: double (nullable = true)
-- regularMarketVolume: double (nullable = true)
-- averageVolume: integer (nullable = true)
-- averageDailyVolume10Day: integer (nullable = true)
-- fiftyDayAverage: double (nullable = true)
-- marketCap: double (nullable = true)
-- Volume: double (nullable = true)
-- payoutRatio_new: double (nullable = true)
-- earningsQuarterlyGrowth_new: double (nullable = true)
-- fiveYearAvgDividendYield_new: double (nullable = true)
-- trailingAnnualDividendRate_new: double (nullable = true)
-- trailingPE_new: double (nullable = true)
-- dividendRate_new: double (nullable = true)
-- dividendYield_new: double (nullable = true)
-- socialScore_new: double (nullable = true)
-- governanceScore_new: double (nullable = true)
-- environmentScore_new: double (nullable = true)
-- fullTimeEmployees_new: double (nullable = true)
-- previousClose_new: double (nullable = true)
-- beta_new: double (nullable = true)
-- pegRatio_new: double (nullable = true)
-- enterpriseToRevenue_new: double (nullable = true)
-- sharesOutstanding_new: double (nullable = true)
-- bookValue_new: double (nullable = true)
-- heldPercentInsiders_new: double (nullable = true)
-- floatShares_new: double (nullable = true)
-- shortPercentOfFloat_new: double (nullable = true)
-- sharesShortPriorMonth_new: double (nullable = true)
-- profitMargins_new: double (nullable = true)
-- trailingEps_new: double (nullable = true)
-- shortRatio_new: double (nullable = true)
-- forwardPE_new: double (nullable = true)
-- previousClose: double (nullable = true)
-- enterpriseToEbitda_new: double (nullable = true)

In [0]:
df_final.printSchema()

root
-- ticker: string (nullable = true)
-- Sector: string (nullable = true)
-- Industry: string (nullable = true)
-- regularMarketOpen: double (nullable = true)
-- regularMarketDayHigh: double (nullable = true)
-- regularMarketDayLow: double (nullable = true)
-- regularMarketPreviousClose: double (nullable = true)
-- regularMarketVolume: double (nullable = true)
-- averageVolume: integer (nullable = true)
-- averageDailyVolume10Day: integer (nullable = true)
-- fiftyDayAverage: double (nullable = true)
-- marketCap: double (nullable = true)
-- Volume: double (nullable = true)
-- payoutRatio_new: double (nullable = true)
-- earningsQuarterlyGrowth_new: double (nullable = true)
-- fiveYearAvgDividendYield_new: double (nullable = true)
-- trailingAnnualDividendRate_new: double (nullable = true)
-- trailingPE_new: double (nullable = true)
-- dividendRate_new: double (nullable = true)
-- dividendYield_new: double (nullable = true)
-- socialScore_new: double (nullable = true)
-- governanceScore_new: double (nullable = true)
-- environmentScore_new: double (nullable = true)
-- fullTimeEmployees_new: double (nullable = true)
-- previousClose_new: double (nullable = true)
-- beta_new: double (nullable = true)
-- pegRatio_new: double (nullable = true)
-- enterpriseToRevenue_new: double (nullable = true)
-- sharesOutstanding_new: double (nullable = true)
-- bookValue_new: double (nullable = true)
-- heldPercentInsiders_new: double (nullable = true)
-- floatShares_new: double (nullable = true)
-- shortPercentOfFloat_new: double (nullable = true)
-- sharesShortPriorMonth_new: double (nullable = true)
-- profitMargins_new: double (nullable = true)
-- trailingEps_new: double (nullable = true)
-- shortRatio_new: double (nullable = true)
-- forwardPE_new: double (nullable = true)
-- previousClose: double (nullable = true)
-- enterpriseToEbitda_new: double (nullable = true)

In [0]:
#numeric columns

numeric_features = [t[0] for t in df_final.dtypes if t[1] == 'double' or t[1] == 'int' ]
# print(df_final.select(numeric_features).describe().toPandas().transpose())
print((numeric_features))

#categorical columns

categoricalcolumns = [t[0] for t in df_final.dtypes if t[1] == 'string']
print(categoricalcolumns)


['regularMarketOpen', 'regularMarketDayHigh', 'regularMarketDayLow', 'regularMarketPreviousClose', 'regularMarketVolume', 'averageVolume', 'averageDailyVolume10Day', 'fiftyDayAverage', 'marketCap', 'Volume', 'payoutRatio_new', 'earningsQuarterlyGrowth_new', 'fiveYearAvgDividendYield_new', 'trailingAnnualDividendRate_new', 'trailingPE_new', 'dividendRate_new', 'dividendYield_new', 'socialScore_new', 'governanceScore_new', 'environmentScore_new', 'fullTimeEmployees_new', 'previousClose_new', 'beta_new', 'pegRatio_new', 'enterpriseToRevenue_new', 'sharesOutstanding_new', 'bookValue_new', 'heldPercentInsiders_new', 'floatShares_new', 'shortPercentOfFloat_new', 'sharesShortPriorMonth_new', 'profitMargins_new', 'trailingEps_new', 'shortRatio_new', 'forwardPE_new', 'previousClose', 'enterpriseToEbitda_new']
['ticker', 'Sector', 'Industry']

In [0]:

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
categorical_columns  = ['Sector']


# Use StringIndexer to convert the categorical columns to hold numerical data
 
stages = []
for categoricalCol in categorical_columns:
  stringIndexer = StringIndexer(inputCol= categoricalCol, outputCol= categoricalCol + '_Index')
  encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],outputCols=[categoricalCol + "_classVec"])
  stages += [stringIndexer,encoder]
  
# Scaling the numeric variables with MinMaxScaler

for standard_scale in numeric_features:
    assembler = VectorAssembler(inputCols= [standard_scale],
                                outputCol=standard_scale +'_vector')
    scaler = MinMaxScaler(inputCol= standard_scale +'_vector', outputCol= standard_scale+ '_scaled')
    stages += [assembler,scaler]

    

pipe = Pipeline(stages=stages)
sparkDF_stocks_final = pipe.fit(df_final).transform(df_final)

In [0]:
sparkDF_stocks_final.show()

+------+--------------------+--------------------+-----------------+--------------------+-------------------+--------------------------+-------------------+-------------+-----------------------+------------------+---------------+-----------+--------------------+---------------------------+----------------------------+------------------------------+------------------+------------------+--------------------+---------------+-------------------+--------------------+---------------------+-----------------+------------------+------------+-----------------------+---------------------+-------------------+-----------------------+---------------+-----------------------+-------------------------+--------------------+-------------------+--------------+------------------+-------------+----------------------+------------+---------------+------------------------+------------------------+---------------------------+---------------------------+--------------------------+--------------------------+---------------------------------+---------------------------------+--------------------------+--------------------------+--------------------+--------------------+------------------------------+------------------------------+----------------------+----------------------+-----------------+--------------------+-------------+--------------------+----------------------+----------------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+-------------------------------------+-------------------------------------+---------------------+---------------------+-----------------------+-----------------------+------------------------+------------------------+----------------------+----------------------+--------------------------+--------------------------+---------------------------+---------------------------+----------------------------+----------------------------+------------------------+------------------------+--------------------+--------------------+-------------------+--------------------+------------------------------+------------------------------+----------------------------+----------------------------+--------------------+--------------------+------------------------------+------------------------------+----------------------+----------------------+------------------------------+------------------------------+--------------------------------+--------------------------------+------------------------+------------------------+----------------------+----------------------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+-----------------------------+-----------------------------+
ticker| Sector| Industry|regularMarketOpen|regularMarketDayHigh|regularMarketDayLow|regularMarketPreviousClose|regularMarketVolume|averageVolume|averageDailyVolume10Day| fiftyDayAverage| marketCap| Volume| payoutRatio_new|earningsQuarterlyGrowth_new|fiveYearAvgDividendYield_new|trailingAnnualDividendRate_new| trailingPE_new| dividendRate_new| dividendYield_new|socialScore_new|governanceScore_new|environmentScore_new|fullTimeEmployees_new|previousClose_new| beta_new|pegRatio_new|enterpriseToRevenue_new|sharesOutstanding_new| bookValue_new|heldPercentInsiders_new|floatShares_new|shortPercentOfFloat_new|sharesShortPriorMonth_new| profitMargins_new| trailingEps_new|shortRatio_new| forwardPE_new|previousClose|enterpriseToEbitda_new|Sector_Index|Sector_classVec|regularMarketOpen_vector|regularMarketOpen_scaled|regularMarketDayHigh_vector|regularMarketDayHigh_scaled|regularMarketDayLow_vector|regularMarketDayLow_scaled|regularMarketPreviousClose_vector|regularMarketPreviousClose_scaled|regularMarketVolume_vector|regularMarketVolume_scaled|averageVolume_vector|averageVolume_scaled|averageDailyVolume10Day_vector|averageDailyVolume10Day_scaled|fiftyDayAverage_vector|fiftyDayAverage_scaled| marketCap_vector| market

In [0]:
sparkDF_stocks_final.columns

Out[25]: ['ticker',
 'Sector',
 'Industry',
 'regularMarketOpen',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
 'averageVolume',
 'averageDailyVolume10Day',
 'fiftyDayAverage',
 'marketCap',
 'Volume',
 'payoutRatio_new',
 'earningsQuarterlyGrowth_new',
 'fiveYearAvgDividendYield_new',
 'trailingAnnualDividendRate_new',
 'trailingPE_new',
 'dividendRate_new',
 'dividendYield_new',
 'socialScore_new',
 'governanceScore_new',
 'environmentScore_new',
 'fullTimeEmployees_new',
 'previousClose_new',
 'beta_new',
 'pegRatio_new',
 'enterpriseToRevenue_new',
 'sharesOutstanding_new',
 'bookValue_new',
 'heldPercentInsiders_new',
 'floatShares_new',
 'shortPercentOfFloat_new',
 'sharesShortPriorMonth_new',
 'profitMargins_new',
 'trailingEps_new',
 'shortRatio_new',
 'forwardPE_new',
 'previousClose',
 'enterpriseToEbitda_new',
 'Sector_Index',
 'Sector_classVec',
 'regularMarketOpen_vector',
 'regularMarketOpen_scaled',
 'regularMarketDayHigh_vector',
 'regularMarketDayHigh_scaled',
 'regularMarketDayLow_vector',
 'regularMarketDayLow_scaled',
 'regularMarketPreviousClose_vector',
 'regularMarketPreviousClose_scaled',
 'regularMarketVolume_vector',
 'regularMarketVolume_scaled',
 'averageVolume_vector',
 'averageVolume_scaled',
 'averageDailyVolume10Day_vector',
 'averageDailyVolume10Day_scaled',
 'fiftyDayAverage_vector',
 'fiftyDayAverage_scaled',
 'marketCap_vector',
 'marketCap_scaled',
 'Volume_vector',
 'Volume_scaled',
 'payoutRatio_new_vector',
 'payoutRatio_new_scaled',
 'earningsQuarterlyGrowth_new_vector',
 'earningsQuarterlyGrowth_new_scaled',
 'fiveYearAvgDividendYield_new_vector',
 'fiveYearAvgDividendYield_new_scaled',
 'trailingAnnualDividendRate_new_vector',
 'trailingAnnualDividendRate_new_scaled',
 'trailingPE_new_vector',
 'trailingPE_new_scaled',
 'dividendRate_new_vector',
 'dividendRate_new_scaled',
 'dividendYield_new_vector',
 'dividendYield_new_scaled',
 'socialScore_new_vector',
 'socialScore_new_scaled',
 'governanceScore_new_vector',
 'governanceScore_new_scaled',
 'environmentScore_new_vector',
 'environmentScore_new_scaled',
 'fullTimeEmployees_new_vector',
 'fullTimeEmployees_new_scaled',
 'previousClose_new_vector',
 'previousClose_new_scaled',
 'beta_new_vector',
 'beta_new_scaled',
 'pegRatio_new_vector',
 'pegRatio_new_scaled',
 'enterpriseToRevenue_new_vector',
 'enterpriseToRevenue_new_scaled',
 'sharesOutstanding_new_vector',
 'sharesOutstanding_new_scaled',
 'bookValue_new_vector',
 'bookValue_new_scaled',
 'heldPercentInsiders_new_vector',
 'heldPercentInsiders_new_scaled',
 'floatShares_new_vector',
 'floatShares_new_scaled',
 'shortPercentOfFloat_new_vector',
 'shortPercentOfFloat_new_scaled',
 'sharesShortPriorMonth_new_vector',
 'sharesShortPriorMonth_new_scaled',
 'profitMargins_new_vector',
 'profitMargins_new_scaled',
 'trailingEps_new_vector',
 'trailingEps_new_scaled',
 'shortRatio_new_vector',
 'shortRatio_new_scaled',
 'forwardPE_new_vector',
 'forwardPE_new_scaled',
 'previousClose_vector',
 'previousClose_scaled',
 'enterpriseToEbitda_new_vector',
 'enterpriseToEbitda_new_scaled']

In [0]:
# scaled variables
sparkDF_stocks_final.select('marketCap',
 'Volume',
 'marketCap_scaled',
 'Volume_scaled').show()

+---------------+-----------+--------------------+--------------------+
 marketCap| Volume| marketCap_scaled| Volume_scaled|
+---------------+-----------+--------------------+--------------------+
3.4188496896E10| 2700586.0|[0.01536597378972...|[0.03399619031575...|
 7.580524544E9|5.8685528E7|[0.00200454230124...|[0.7433287510795208]|
 9.793439744E9| 1312474.0|[0.00311577747705...|[0.01640872832307...|
 1.99499E12|7.3604287E7|[0.9999999999999999]|[0.9323503191448683]|
 1.77965E11| 9682935.0|[0.08756463178831...|[0.12246296940639...|
2.0508104704E10| 913807.0|[0.00849624239528...|[0.01135759360899...|
 1.96537E11| 4075207.0|[0.09689072782905...|[0.05141272053220...|
 1.54454E11| 1542330.0|[0.07575837268784...|[0.01932101757912...|
 2.22072E11| 1851107.0|[0.10971335664664...|[0.02323324062143...|
4.9846255616E10| 2295742.0|[0.02322865748590...|[0.02886679264253...|
2.7046068224E10| 1738342.0|[0.01177933929913...|[0.02180450133267...|
7.3442975744E10| 1394982.0|[0.03507796171594...|[0.01745410961370...|
5.5892025344E10| 1458524.0|[0.02626459487140...|[0.01825919055093...|
1.9393388544E10| 1358445.0|[0.00793647769754...|[0.01699118364027...|
4.0947212288E10| 5841366.0|[0.01875992322208...|[0.07379006060672...|
1.3581975552E10| 6956379.0|[0.00501822464479...|[0.08791734194917...|
3.0802210816E10| 4181350.0| [0.013665519923502]|[0.05275755868524...|
3.3263519744E10| 5571857.0|[0.01490148823324...|[0.07037536547104...|
 7.86726912E9| 416977.0|[0.00214853365677...|[0.00506272781876...|
 2.222280704E10| 1902762.0|[0.00935729552186...|[0.02388771255830...|
+---------------+-----------+--------------------+--------------------+
only showing top 20 rows

In [0]:
assemblerInputs = [
 'Sector_classVec',
#  'Industry_classVec',
#  'previousClose_classVec',
 'regularMarketOpen_scaled',
#  'twoHundredDayAverage_scaled',
 'regularMarketDayHigh_scaled',
 'regularMarketDayLow_scaled',
 'regularMarketPreviousClose_scaled',
#  'regularMarketVolume_scaled',
 'averageVolume_scaled',
 'averageDailyVolume10Day_scaled',
#  'fiftyTwoWeekHigh_scaled',
#  'fiftyTwoWeekLow_scaled',
 'fiftyDayAverage_scaled',
 'marketCap_scaled',
 'Volume_scaled',
 'payoutRatio_new_scaled',
 'earningsQuarterlyGrowth_new_scaled',
 'fiveYearAvgDividendYield_new_scaled',
 'trailingAnnualDividendRate_new_scaled',
 'trailingPE_new_scaled',
 'dividendRate_new_scaled',
 'dividendYield_new_scaled',
 'socialScore_new_scaled',
 'governanceScore_new_scaled',
 'environmentScore_new_scaled',
 'fullTimeEmployees_new_scaled',
 'previousClose_new_scaled',
 'beta_new_scaled',
 'pegRatio_new_scaled',
 'enterpriseToRevenue_new_scaled',
 'sharesOutstanding_new_scaled',
 'bookValue_new_scaled',
 'heldPercentInsiders_new_scaled',
 'floatShares_new_scaled',
 'shortPercentOfFloat_new_scaled',
 'sharesShortPriorMonth_new_scaled',
 'profitMargins_new_scaled',
 'trailingEps_new_scaled',
 'shortRatio_new_scaled',
 'forwardPE_new_scaled',
 'enterpriseToEbitda_new_scaled']

In [0]:
#numeric columns

numeric_features = [t[0] for t in sparkDF_stocks_final.dtypes if t[1] == 'double' or t[1] == 'int' ]
# print(df_final.select(numeric_features).describe().toPandas().transpose())
print((numeric_features))

#categorical columns

categoricalcolumns = [t[0] for t in sparkDF_stocks_final.dtypes if t[1] == 'string']
print(categoricalcolumns)

vector_columns = [t[0] for t in sparkDF_stocks_final.dtypes if t[1] == 'vector']
print(vector_columns)


['regularMarketOpen', 'regularMarketDayHigh', 'regularMarketDayLow', 'regularMarketPreviousClose', 'regularMarketVolume', 'averageVolume', 'averageDailyVolume10Day', 'fiftyDayAverage', 'marketCap', 'Volume', 'payoutRatio_new', 'earningsQuarterlyGrowth_new', 'fiveYearAvgDividendYield_new', 'trailingAnnualDividendRate_new', 'trailingPE_new', 'dividendRate_new', 'dividendYield_new', 'socialScore_new', 'governanceScore_new', 'environmentScore_new', 'fullTimeEmployees_new', 'previousClose_new', 'beta_new', 'pegRatio_new', 'enterpriseToRevenue_new', 'sharesOutstanding_new', 'bookValue_new', 'heldPercentInsiders_new', 'floatShares_new', 'shortPercentOfFloat_new', 'sharesShortPriorMonth_new', 'profitMargins_new', 'trailingEps_new', 'shortRatio_new', 'forwardPE_new', 'previousClose', 'enterpriseToEbitda_new', 'Sector_Index']
['ticker', 'Sector', 'Industry']
['Sector_classVec', 'regularMarketOpen_vector', 'regularMarketOpen_scaled', 'regularMarketDayHigh_vector', 'regularMarketDayHigh_scaled', 'regularMarketDayLow_vector', 'regularMarketDayLow_scaled', 'regularMarketPreviousClose_vector', 'regularMarketPreviousClose_scaled', 'regularMarketVolume_vector', 'regularMarketVolume_scaled', 'averageVolume_vector', 'averageVolume_scaled', 'averageDailyVolume10Day_vector', 'averageDailyVolume10Day_scaled', 'fiftyDayAverage_vector', 'fiftyDayAverage_scaled', 'marketCap_vector', 'marketCap_scaled', 'Volume_vector', 'Volume_scaled', 'payoutRatio_new_vector', 'payoutRatio_new_scaled', 'earningsQuarterlyGrowth_new_vector', 'earningsQuarterlyGrowth_new_scaled', 'fiveYearAvgDividendYield_new_vector', 'fiveYearAvgDividendYield_new_scaled', 'trailingAnnualDividendRate_new_vector', 'trailingAnnualDividendRate_new_scaled', 'trailingPE_new_vector', 'trailingPE_new_scaled', 'dividendRate_new_vector', 'dividendRate_new_scaled', 'dividendYield_new_vector', 'dividendYield_new_scaled', 'socialScore_new_vector', 'socialScore_new_scaled', 'governanceScore_new_vector', 'governanceScore_new_scaled', 'environmentScore_new_vector', 'environmentScore_new_scaled', 'fullTimeEmployees_new_vector', 'fullTimeEmployees_new_scaled', 'previousClose_new_vector', 'previousClose_new_scaled', 'beta_new_vector', 'beta_new_scaled', 'pegRatio_new_vector', 'pegRatio_new_scaled', 'enterpriseToRevenue_new_vector', 'enterpriseToRevenue_new_scaled', 'sharesOutstanding_new_vector', 'sharesOutstanding_new_scaled', 'bookValue_new_vector', 'bookValue_new_scaled', 'heldPercentInsiders_new_vector', 'heldPercentInsiders_new_scaled', 'floatShares_new_vector', 'floatShares_new_scaled', 'shortPercentOfFloat_new_vector', 'shortPercentOfFloat_new_scaled', 'sharesShortPriorMonth_new_vector', 'sharesShortPriorMonth_new_scaled', 'profitMargins_new_vector', 'profitMargins_new_scaled', 'trailingEps_new_vector', 'trailingEps_new_scaled', 'shortRatio_new_vector', 'shortRatio_new_scaled', 'forwardPE_new_vector', 'forwardPE_new_scaled', 'previousClose_vector', 'previousClose_scaled', 'enterpriseToEbitda_new_vector', 'enterpriseToEbitda_new_scaled']

In [0]:
sparkDF_stocks_final.printSchema()

root
-- ticker: string (nullable = true)
-- Sector: string (nullable = true)
-- Industry: string (nullable = true)
-- regularMarketOpen: double (nullable = true)
-- regularMarketDayHigh: double (nullable = true)
-- regularMarketDayLow: double (nullable = true)
-- regularMarketPreviousClose: double (nullable = true)
-- regularMarketVolume: double (nullable = true)
-- averageVolume: integer (nullable = true)
-- averageDailyVolume10Day: integer (nullable = true)
-- fiftyDayAverage: double (nullable = true)
-- marketCap: double (nullable = true)
-- Volume: double (nullable = true)
-- payoutRatio_new: double (nullable = true)
-- earningsQuarterlyGrowth_new: double (nullable = true)
-- fiveYearAvgDividendYield_new: double (nullable = true)
-- trailingAnnualDividendRate_new: double (nullable = true)
-- trailingPE_new: double (nullable = true)
-- dividendRate_new: double (nullable = true)
-- dividendYield_new: double (nullable = true)
-- socialScore_new: double (nullable = true)
-- governanceScore_new: double (nullable = true)
-- environmentScore_new: double (nullable = true)
-- fullTimeEmployees_new: double (nullable = true)
-- previousClose_new: double (nullable = true)
-- beta_new: double (nullable = true)
-- pegRatio_new: double (nullable = true)
-- enterpriseToRevenue_new: double (nullable = true)
-- sharesOutstanding_new: double (nullable = true)
-- bookValue_new: double (nullable = true)
-- heldPercentInsiders_new: double (nullable = true)
-- floatShares_new: double (nullable = true)
-- shortPercentOfFloat_new: double (nullable = true)
-- sharesShortPriorMonth_new: double (nullable = true)
-- profitMargins_new: double (nullable = true)
-- trailingEps_new: double (nullable = true)
-- shortRatio_new: double (nullable = true)
-- forwardPE_new: double (nullable = true)
-- previousClose: double (nullable = true)
-- enterpriseToEbitda_new: double (nullable = true)
-- Sector_Index: double (nullable = false)
-- Sector_classVec: vector (nullable = true)
-- regularMarketOpen_vector: vector (nullable = true)
-- regularMarketOpen_scaled: vector (nullable = true)
-- regularMarketDayHigh_vector: vector (nullable = true)
-- regularMarketDayHigh_scaled: vector (nullable = true)
-- regularMarketDayLow_vector: vector (nullable = true)
-- regularMarketDayLow_scaled: vector (nullable = true)
-- regularMarketPreviousClose_vector: vector (nullable = true)
-- regularMarketPreviousClose_scaled: vector (nullable = true)
-- regularMarketVolume_vector: vector (nullable = true)
-- regularMarketVolume_scaled: vector (nullable = true)
-- averageVolume_vector: vector (nullable = true)
-- averageVolume_scaled: vector (nullable = true)
-- averageDailyVolume10Day_vector: vector (nullable = true)
-- averageDailyVolume10Day_scaled: vector (nullable = true)
-- fiftyDayAverage_vector: vector (nullable = true)
-- fiftyDayAverage_scaled: vector (nullable = true)
-- marketCap_vector: vector (nullable = true)
-- marketCap_scaled: vector (nullable = true)
-- Volume_vector: vector (nullable = true)
-- Volume_scaled: vector (nullable = true)
-- payoutRatio_new_vector: vector (nullable = true)
-- payoutRatio_new_scaled: vector (nullable = true)
-- earningsQuarterlyGrowth_new_vector: vector (nullable = true)
-- earningsQuarterlyGrowth_new_scaled: vector (nullable = true)
-- fiveYearAvgDividendYield_new_vector: vector (nullable = true)
-- fiveYearAvgDividendYield_new_scaled: vector (nullable = true)
-- trailingAnnualDividendRate_new_vector: vector (nullable = true)
-- trailingAnnualDividendRate_new_scaled: vector (nullable = true)
-- trailingPE_new_vector: vector (nullable = true)
-- trailingPE_new_scaled: vector (nullable = true)
-- dividendRate_new_vector: vector (nullable = true)
-- dividendRate_new_scaled: vector (nullable = true)
-- dividendYield_new_vector: vector (nullable = true)
-- dividendYield_new_scaled: vector (nullable = true)
-- socialScore_new_vector: vector (nullable = true)
-- socialScore_new_scaled: vector (nullable = true)
-- governanceScore_new_vector: v

In [0]:

stages = []
assembler = VectorAssembler(inputCols=assemblerInputs,outputCol="features")
stages += [assembler]


pipe = Pipeline(stages=stages)
sparkDF_stocks_final_1 =pipe.fit(sparkDF_stocks_final).transform(sparkDF_stocks_final)

In [0]:
sparkDF_stocks_final_1.select('features').collect()

Out[31]: [Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0037, 0.0038, 0.0038, 0.0038, 0.0268, 0.0318, 0.0037, 0.0024, 0.0376, 0.0336, 0.0506, 0.5167, 0.0705, 0.0014, 0.0702, 0.415, 0.3892, 0.1984, 0.0016, 0.0246, 0.0038, 0.1873, 0.4757, 0.1971, 0.0226, 0.1021, 0.011, 0.0226, 0.098, 0.0999, 0.5534, 0.1989, 0.1235, 0.017, 0.8386])),
 Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0126, 0.0128, 0.0129, 0.0128, 0.0154, 0.0269, 0.0117, 0.0046, 0.0206, 0.0287, 0.0318, 0.4658, 0.1832, 0.0013, 0.1791, 0.3841, 0.3937, 0.3486, 0.0023, 0.0317, 0.0128, 0.141, 0.4676, 0.1846, 0.0124, 0.1082, 0.0132, 0.0124, 0.0887, 0.0932, 0.554, 0.21, 0.164, 0.0168, 0.8369])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.047, 0.0474, 0.0467, 0.0475, 0.151, 0.3229, 0.0395, 0.0548, 0.2278, 0.0083, 0.0224, 0.377, 0.2896, 0.0027, 0.2242, 0.0863, 0.8749, 0.5019, 0.312, 0.0732, 0.0475, 0.2943, 0.4621, 0.2263, 0.0328, 0.0699, 0.0018, 0.0312, 0.0263, 0.0619, 0.5083, 0.1644, 0.0, 0.0407, 0.8009])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0876, 0.0877, 0.0874, 0.0881, 0.0078, 0.0102, 0.0877, 0.0499, 0.0139, 0.02, 0.0311, 0.3697, 0.6765, 0.0015, 0.7163, 0.2493, 0.6631, 0.2669, 0.2785, 0.0499, 0.0881, 0.21, 0.4658, 0.202, 0.0161, 0.1145, 0.0011, 0.0138, 0.0114, 0.0166, 0.5638, 0.2795, 0.0737, 0.0174, 0.8398])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0717, 0.0719, 0.0723, 0.0722, 0.0055, 0.0069, 0.0724, 0.0239, 0.0096, 0.0184, 0.0311, 0.2198, 0.3904, 0.002, 0.3994, 0.1665, 0.7015, 0.1572, 0.344, 0.0408, 0.0722, 0.1704, 0.4657, 0.2038, 0.0095, 0.1665, 0.0075, 0.0096, 0.0168, 0.0079, 0.5535, 0.2476, 0.0507, 0.0171, 0.8425])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1383, 0.1395, 0.1365, 0.139, 0.0026, 0.0038, 0.1211, 0.0139, 0.0043, 0.0, 0.0057, 0.1985, 0.2724, 0.0061, 0.2242, 0.0863, 0.8472, 0.4623, 0.4449, 0.0064, 0.139, 0.3351, 0.3869, 0.4732, 0.003, 0.0098, 0.0204, 0.003, 0.0975, 0.0176, 0.5749, 0.2268, 0.2325, 0.0215, 0.8493])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0093, 0.0093, 0.0094, 0.0094, 0.0085, 0.0111, 0.0082, 0.0033, 0.0141, 0.0033, 0.0146, 0.0262, 0.0056, 0.0036, 0.0055, 0.0159, 0.8222, 0.2965, 0.4277, 0.0146, 0.0094, 0.3534, 0.4573, 0.1771, 0.0132, 0.123, 0.0071, 0.0132, 0.0356, 0.039, 0.5387, 0.1979, 0.1167, 0.0179, 0.8401])),
 Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0063, 0.0066, 0.0064, 0.0063, 0.0157, 0.0293, 0.0061, 0.0018, 0.0348, 0.044, 0.0317, 0.5007, 0.0846, 0.0024, 0.0826, 0.3154, 0.3534, 0.1424, 0.7203, 0.0013, 0.0063, 0.2303, 0.4591, 0.259, 0.0123, 0.1093, 0.0082, 0.0123, 0.0429, 0.031, 0.5537, 0.1985, 0.0698, 0.0198, 0.8373])),
 Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.026, 0.0259, 0.026, 0.026, 0.0041, 0.0057, 0.0248, 0.0056, 0.0072, 0.0196, 0.0365, 0.1863, 0.124, 0.0028, 0.1212, 0.1366, 0.5474, 0.3689, 0.7137, 0.0028, 0.026, 0.2185, 0.4657, 0.2771, 0.0074, 0.1203, 0.0108, 0.0074, 0.0082, 0.0075, 0.5652, 0.2077, 0.0481, 0.0177, 0.842])),
 Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0033, 0.0033, 0.0034, 0.0033, 0.0307, 0.0343, 0.0035, 0.0019, 0.0388, 0.0212, 0.0317, 0.3246, 0.0141, 0.0018, 0.0138, 0.0899, 0.4736, 0.3953, 0.573, 0.0056, 0.0033, 0.3828, 0.4707, 0.1896, 0.022, 0.1201, 0.1534, 0.02, 0.0428, 0.0397, 0.4902, 0.183, 0.0737, 0.018, 0.8391])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0017, 0.0017, 0.0017, 0.0017, 0.4308, 0.7023, 0.002, 0.002, 0.7433, 0.0, 0.0293, 0.1863, 0.0141, 0.0, 0.0, 0.0, 0.576, 0.1712, 0.4254, 0.0502, 0.0017, 0.3578, 0.464, 0.1966, 0.0329, 0.0815, 0.0164, 0.0294, 0.8568, 1.0, 0.4506, 0.1417, 0.0963, 0.0142, 0.8233])),
 Row(features=DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0076, 0.0076, 0.00

## Machine learning Models:

In [0]:
from pyspark.mllib.linalg import Vectors, VectorUDT
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql.functions import udf

update_vector = udf(lambda vector: Vectors.dense(vector), VectorUDT())
df_stocks_final = sparkDF_stocks_final_1.withColumn('features',update_vector(sparkDF_stocks_final_1.features))
df_stocks_final.select('features').show()

+--------------------+
 features|
+--------------------+
[0.0,0.0,0.0,0.0,...|
[0.0,0.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,0.0,0.0,0.0,...|
[0.0,0.0,0.0,0.0,...|
[0.0,0.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,1.0,0.0,0.0,...|
[0.0,0.0,1.0,0.0,...|
[0.0,0.0,1.0,0.0,...|
[0.0,0.0,1.0,0.0,...|
[0.0,0.0,1.0,0.0,...|
[0.0,0.0,1.0,0.0,...|
[0.0,0.0,1.0,0.0,...|
+--------------------+
only showing top 20 rows

### Cosine similarity model

In [0]:
def sim_cos(v1,v2):
    try:
        p = 2
        return float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
    except:
        return 0

In [0]:
# calculating cosine similiarity for sample two rows
v1 = df_stocks_final.select('features').collect()[50][0]
v2 = df_stocks_final.select('features').collect()[20][0]
print(v1)
print(v2)
print('cosine similiarity of v1 and v2 vectors:' + str(sim_cos(v1,v2)))

[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0022768487278728372,0.002264895187481547,0.0022849614789757005,0.002320753282785094,0.055166071990805025,0.07226464287819617,0.0023420865290216058,0.0054854000377931155,0.07769554317724504,0.03828292926829268,0.038478166882836146,0.42794759825327505,0.04369274136715997,0.0018244892072588345,0.04269972451790634,0.36123342191776026,0.3909740840035746,0.34552529182879377,0.05765485001947799,0.008905708408623413,0.002320753282785094,0.3411845961656118,0.4630834567131613,0.2712675341621661,0.055952742535556224,0.11361005743511861,0.006563675658368687,0.056144781539557646,0.035407537248028045,0.13386956642038422,0.5904757707543355,0.19649801124369487,0.07793867120954003,0.016769210259792788,0.8295139297993811]
[0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008057632881194838,0.008281074697372307,0.008056479808130797,0.008043061018946088,0.028593807241253242,0.06586865139071918,0.007100358130511746,0.0037755057761169677,0.0896638970169434,0.021489024390243902,0.024349947501698475,0.8107714701601164,0.042283298097251586,0.0055028426456542505,0.0,0.0,0.2823949955317247,0.245136186770428,0.09622126996493963,0.011512279385860238,0.008043061018946088,0.3453411350282158,0.4668919971947323,0.19602015295951283,0.016030884744889517,0.08616306403721452,0.30758024493716485,0.0096920432330934,0.07099036459246276,0.07147720102198071,0.5259325668382773,0.190765060107104,0.075809199318569,0.017766694541538893,0.8383300664445897]
cosine similiarity of v1 and v2 vectors:0.6270628187072109

In [0]:
# calculating cosine similiary for Boeing('BA') with all other ticker

def sim_cos(v1,v2):
    try:
        p = 2
        return float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
    except:
        return 0
      
cosine_list= []

v1 = df_stocks_final.select('features').collect()[46][0]

for j in range(0,df_stocks_final.count()):
      v2 = df_stocks_final.select('features').collect()[j][0]
      cosine_list.append((df_stocks_final.select('ticker').collect()[46][0],df_stocks_final.select('ticker').collect()[j][0],sim_cos(v1,v2)))

from pyspark.sql import Row
similiar_columns = ["ticker_1", "ticker_2", "cosine_similiarity"]
df_cosine_similiarity = spark.createDataFrame(data=cosine_list, schema = similiar_columns)
# df_cosine_similiarity.write.option.csv("dbfs:/FileStore/shared_uploads/baradi@usf.edu/cosine.csv")
# df_cosine_similiarity.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df_cosine_AAPL/Sample_cosine_AAPL.csv")
      

In [0]:
df_cosine_simliarity_csv = spark.read.format("csv").load("dbfs:/FileStore/df_cosine_AAPL/Sample_cosine_AAPL.csv",inferSchema=True,header=True)
df_cosine_simliarity_csv.sort('cosine_similiarity', ascending = False).show(10)

+--------+--------+------------------+
ticker_1|ticker_2|cosine_similiarity|
+--------+--------+------------------+
 BA| BA| 1.0|
 BA| AWK|0.9977401761824249|
 BA| AVGO|0.9952305828034876|
 BA| AZO|0.9945060179258373|
 BA| BIIB|0.9931214657044998|
 BA| BAX|0.9895375910632066|
 BA| BDX|0.9889141705471834|
 BA| IBM|0.9879624664166052|
 BA| HPQ|0.9856935669214587|
 BA| AVY|0.9855526932485693|
+--------+--------+------------------+
only showing top 10 rows

## Creating matrix form

In [0]:
from pyspark.sql.functions import monotonically_increasing_id 

df_stocks_final = df_stocks_final.select("*").withColumn("id", monotonically_increasing_id())
df_stocks_final_check = df_stocks_final.sort('id').limit(10)

In [0]:
# creating id column

df_stocks_final = df_stocks_final.select("*").withColumn("id", monotonically_increasing_id())
print(df_stocks_final.show())

#creating 'stocks_matrix' form for all tickers

from pyspark.mllib.linalg.distributed import IndexedRowMatrix
mat = IndexedRowMatrix(df_stocks_final.select('id','features')).toBlockMatrix()
dot = mat.multiply(mat.transpose())
stocks_matrix = dot.toLocalMatrix().toArray()
print(stocks_matrix)


+------+--------------------+--------------------+-----------------+--------------------+-------------------+--------------------------+-------------------+-------------+-----------------------+------------------+---------------+-----------+--------------------+---------------------------+----------------------------+------------------------------+------------------+------------------+--------------------+---------------+-------------------+--------------------+---------------------+-----------------+------------------+------------+-----------------------+---------------------+-------------------+-----------------------+---------------+-----------------------+-------------------------+--------------------+-------------------+--------------+------------------+-------------+----------------------+------------+---------------+------------------------+------------------------+---------------------------+---------------------------+--------------------------+--------------------------+---------------------------------+---------------------------------+--------------------------+--------------------------+--------------------+--------------------+------------------------------+------------------------------+----------------------+----------------------+-----------------+--------------------+-------------+--------------------+----------------------+----------------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+-------------------------------------+-------------------------------------+---------------------+---------------------+-----------------------+-----------------------+------------------------+------------------------+----------------------+----------------------+--------------------------+--------------------------+---------------------------+---------------------------+----------------------------+----------------------------+------------------------+------------------------+--------------------+--------------------+-------------------+--------------------+------------------------------+------------------------------+----------------------------+----------------------------+--------------------+--------------------+------------------------------+------------------------------+----------------------+----------------------+------------------------------+------------------------------+--------------------------------+--------------------------------+------------------------+------------------------+----------------------+----------------------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+-----------------------------+-----------------------------+--------------------+---+
ticker| Sector| Industry|regularMarketOpen|regularMarketDayHigh|regularMarketDayLow|regularMarketPreviousClose|regularMarketVolume|averageVolume|averageDailyVolume10Day| fiftyDayAverage| marketCap| Volume| payoutRatio_new|earningsQuarterlyGrowth_new|fiveYearAvgDividendYield_new|trailingAnnualDividendRate_new| trailingPE_new| dividendRate_new| dividendYield_new|socialScore_new|governanceScore_new|environmentScore_new|fullTimeEmployees_new|previousClose_new| beta_new|pegRatio_new|enterpriseToRevenue_new|sharesOutstanding_new| bookValue_new|heldPercentInsiders_new|floatShares_new|shortPercentOfFloat_new|sharesShortPriorMonth_new| profitMargins_new| trailingEps_new|shortRatio_new| forwardPE_new|previousClose|enterpriseToEbitda_new|Sector_Index|Sector_classVec|regularMarketOpen_vector|regularMarketOpen_scaled|regularMarketDayHigh_vector|regularMarketDayHigh_scaled|regularMarketDayLow_vector|regularMarketDayLow_scaled|regularMarketPreviousClose_vector|regularMarketPreviousClose_scaled|regularMarketVolume_vector|regularMarketVolume_scaled|averageVolume_vector|averageVolume_scaled|averageDailyVolume10Day_vector|averageDailyVolume10Day_scaled|fiftyDayAverage_vector|fiftyDayAverage_scaled|

In [0]:
# df_check  = sc.parallelize(stocks_matrix).toDF(ticker_list)
stocks_matrix_new = (stocks_matrix)*1000

In [0]:
df_stocks_final.select('ticker').show()

In [0]:
# scaling the cosine similiarity values 
stocks_matrix_new = (stocks_matrix)*1000

# checking the length of ticker_list created, initial DF, and stock_matrix(all should be same size)

print(stocks_matrix_new.shape)
print(df.count())

In [0]:
# creating list of tickers as columns names for matrix

ticker_list =[]
for i in df_stocks_final.select('ticker').collect():
  ticker_list.append(i[0])
ticker_list

In [0]:
df_stocks_final.select('features').collect()[0]

In [0]:
#converting matrix to dataframe with ticker list as column names

rdd_spark = sc.parallelize(stocks_matrix_new)
rdd_spark2 = rdd_spark.map(lambda x: [int(i) for i in x])
df_stock_matrix = rdd_spark2.toDF(ticker_list)
df_stock_matrix.show()

In [0]:
# columns for matrix
col=[]

for i in df_stocks_final.select('ticker').collect():
  L=[]
  L.append(i[0])
  col.append(L)

In [0]:
# creating empty dataframe with 'ticker_symbols' and 'id' columns

from pyspark.sql import Row
l = sc.parallelize(ticker_list)
new_row = Row("ticker_name")
df_ticker_names = l.map(new_row).toDF()

print(df_ticker_names.show(3))

In [0]:
# creating common column 'id' to join the df_stock_matrix and l_as_df to add ticker_name column
from pyspark.sql.functions import monotonically_increasing_id 

df_stock_recommendation = df_stock_matrix.select("*").withColumn("id", monotonically_increasing_id())
df_ticker_names = df_ticker_names.select("*").withColumn("id", monotonically_increasing_id())
print(df_stock_recommendation.select('id').show(4),df_ticker_names.select('id').show(4))


In [0]:
# creating ticker column in df_stock_recommendation data

df_final_stock_recommendation = df_stock_recommendation.join(df_ticker_names, on = 'id',how = 'left')
df_final_stock_recommendation.show()

In [0]:
df_final_stock_recommendation.select('ticker_name', 'NFLX').sort('NFLX',ascending=False).show(10)

In [0]:
#creating recommendation for particular ticker

df_final_stock_recommendation.select('ticker_name','bkng').sort('bkng',ascending=False).show(10)                                    

In [0]:
df_final_stock_recommendation_renamed = df_final_stock_recommendation.withColumnRenamed('ticker_name','ticker')
df_final_stock_recommendation_renamed.select('ticker').show(3)

In [0]:
df_stocks_final_merged.columns

In [0]:
# df = df_stocks_final.selectExpr("ticker_name", "ticker")

df_stocks_final_merged = df.join(df_final_stock_recommendation_renamed, on = 'ticker', how = 'inner')

#creating recommendation for particular ticker

df_stocks_final_merged.select('ticker','shortName','sector','industry','AAPL').sort('AAPL',ascending=False                                                              ).show(10)
                                                            


### K-means model:

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

for i in range(2,20):
  kmeans = KMeans().setK(i).setSeed(999)
  model = kmeans.fit(final_data2)
  evaluator = ClusteringEvaluator()
  predictions = model.transform(final_data2)
  silhouette = evaluator.evaluate(predictions)
  print("training Dataset Performance for cluster : " + str(i)+ " is " + str(silhouette))


In [0]:
# Kmeans custering with 5 clusters

# train, test = final_data2.randomSplit([0.8, 0.2], seed = 99999)
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans = KMeans().setK(10).setSeed(999)
model = kmeans.fit(sparkDF_stocks_final_1)
evaluator = ClusteringEvaluator()
predictions = model.transform(sparkDF_stocks_final_1)
silhouette = evaluator.evaluate(predictions)
print('with clusters : ' + str(10))
print("training Dataset Performance = " + str(silhouette))

  
transformed = model.transform(sparkDF_stocks_final_1)
transformed.show()

with clusters : 10
training Dataset Performance = 0.7260441273736291
+------+--------------------+--------------------+-----------------+--------------------+-------------------+--------------------------+-------------------+-------------+-----------------------+------------------+---------------+-----------+--------------------+---------------------------+----------------------------+------------------------------+------------------+------------------+--------------------+---------------+-------------------+--------------------+---------------------+-----------------+------------------+------------+-----------------------+---------------------+-------------------+-----------------------+---------------+-----------------------+-------------------------+--------------------+-------------------+--------------+------------------+-------------+----------------------+------------+---------------+------------------------+------------------------+---------------------------+---------------------------+--------------------------+--------------------------+---------------------------------+---------------------------------+--------------------------+--------------------------+--------------------+--------------------+------------------------------+------------------------------+----------------------+----------------------+-----------------+--------------------+-------------+--------------------+----------------------+----------------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+-------------------------------------+-------------------------------------+---------------------+---------------------+-----------------------+-----------------------+------------------------+------------------------+----------------------+----------------------+--------------------------+--------------------------+---------------------------+---------------------------+----------------------------+----------------------------+------------------------+------------------------+--------------------+--------------------+-------------------+--------------------+------------------------------+------------------------------+----------------------------+----------------------------+--------------------+--------------------+------------------------------+------------------------------+----------------------+----------------------+------------------------------+------------------------------+--------------------------------+--------------------------------+------------------------+------------------------+----------------------+----------------------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+-----------------------------+-----------------------------+--------------------+----------+
ticker| Sector| Industry|regularMarketOpen|regularMarketDayHigh|regularMarketDayLow|regularMarketPreviousClose|regularMarketVolume|averageVolume|averageDailyVolume10Day| fiftyDayAverage| marketCap| Volume| payoutRatio_new|earningsQuarterlyGrowth_new|fiveYearAvgDividendYield_new|trailingAnnualDividendRate_new| trailingPE_new| dividendRate_new| dividendYield_new|socialScore_new|governanceScore_new|environmentScore_new|fullTimeEmployees_new|previousClose_new| beta_new|pegRatio_new|enterpriseToRevenue_new|sharesOutstanding_new| bookValue_new|heldPercentInsiders_new|floatShares_new|shortPercentOfFloat_new|sharesShortPriorMonth_new| profitMargins_new| trailingEps_new|shortRatio_new| forwardPE_new|previousClose|enterpriseToEbitda_new|Sector_Index|Sector_classVec|regularMarketOpen_vector|regularMarketOpen_scaled|regularMarketDayHigh_vector|regularMarketDayHigh_scaled|regularMarketDayLow_vector|regularMarketDayLow_scaled|regularMarketPreviousClose_vector|regularMarketPreviousClose_scaled|regularMarketVolume_vector|regularMarketVolume_scaled|averageVolume_vector|averageVolume_scaled|averageDailyVolume10Day_vector|a

In [0]:
# applying PCA to feeatures to create 2D-plot
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model_df = pca.fit(transformed).transform(transformed)

In [0]:
model_df.select('pcaFeatures').collect()

Out[18]: [Row(pcaFeatures=DenseVector([-0.1245, -0.155])),
 Row(pcaFeatures=DenseVector([-0.181, -0.1437])),
 Row(pcaFeatures=DenseVector([-0.019, 0.0231])),
 Row(pcaFeatures=DenseVector([0.0622, -0.0604])),
 Row(pcaFeatures=DenseVector([0.1105, 0.0297])),
 Row(pcaFeatures=DenseVector([0.0377, 0.095])),
 Row(pcaFeatures=DenseVector([0.0725, 0.1423])),
 Row(pcaFeatures=DenseVector([0.1063, -0.2898])),
 Row(pcaFeatures=DenseVector([-0.0096, -0.1263])),
 Row(pcaFeatures=DenseVector([-0.0262, -0.1586])),
 Row(pcaFeatures=DenseVector([0.2025, -0.0637])),
 Row(pcaFeatures=DenseVector([0.1856, -0.0091])),
 Row(pcaFeatures=DenseVector([0.115, 0.0868])),
 Row(pcaFeatures=DenseVector([0.168, 0.0468])),
 Row(pcaFeatures=DenseVector([0.2558, -0.3418])),
 Row(pcaFeatures=DenseVector([0.2498, -0.2132])),
 Row(pcaFeatures=DenseVector([0.2628, -0.2499])),
 Row(pcaFeatures=DenseVector([0.2238, -0.2138])),
 Row(pcaFeatures=DenseVector([0.2902, -0.2728])),
 Row(pcaFeatures=DenseVector([0.2948, -0.3345])),
 Row(pcaFeatures=DenseVector([0.2782, -0.3671])),
 Row(pcaFeatures=DenseVector([0.2375, -0.1407])),
 Row(pcaFeatures=DenseVector([0.2697, -0.1764])),
 Row(pcaFeatures=DenseVector([-1.037, -0.4902])),
 Row(pcaFeatures=DenseVector([-0.8977, -0.5565])),
 Row(pcaFeatures=DenseVector([-1.0055, -0.4847])),
 Row(pcaFeatures=DenseVector([-1.0095, -0.5272])),
 Row(pcaFeatures=DenseVector([-0.9948, -0.6111])),
 Row(pcaFeatures=DenseVector([-1.0026, -0.4564])),
 Row(pcaFeatures=DenseVector([-0.9485, -0.4717])),
 Row(pcaFeatures=DenseVector([-0.9231, -0.4824])),
 Row(pcaFeatures=DenseVector([0.2643, -0.1001])),
 Row(pcaFeatures=DenseVector([0.2202, -0.4634])),
 Row(pcaFeatures=DenseVector([0.2535, -0.3428])),
 Row(pcaFeatures=DenseVector([0.2813, -0.2082])),
 Row(pcaFeatures=DenseVector([0.2206, -0.2263])),
 Row(pcaFeatures=DenseVector([0.3386, -0.204])),
 Row(pcaFeatures=DenseVector([-1.0316, -0.4598])),
 Row(pcaFeatures=DenseVector([-1.0854, -0.4952])),
 Row(pcaFeatures=DenseVector([-0.9666, -0.5036])),
 Row(pcaFeatures=DenseVector([-1.135, -0.5158])),
 Row(pcaFeatures=DenseVector([-1.0225, -0.5077])),
 Row(pcaFeatures=DenseVector([-1.0547, -0.5358])),
 Row(pcaFeatures=DenseVector([-0.9914, -0.497])),
 Row(pcaFeatures=DenseVector([-1.007, -0.3277])),
 Row(pcaFeatures=DenseVector([-0.9279, -0.5494])),
 Row(pcaFeatures=DenseVector([-0.9997, -0.5245])),
 Row(pcaFeatures=DenseVector([-1.0576, -0.4932])),
 Row(pcaFeatures=DenseVector([-0.9721, -0.5807])),
 Row(pcaFeatures=DenseVector([-0.9435, -0.4905])),
 Row(pcaFeatures=DenseVector([-0.9015, -0.5178])),
 Row(pcaFeatures=DenseVector([-1.0456, -0.4655])),
 Row(pcaFeatures=DenseVector([-1.0561, -0.4755])),
 Row(pcaFeatures=DenseVector([0.0211, -0.1736])),
 Row(pcaFeatures=DenseVector([0.0289, -0.2394])),
 Row(pcaFeatures=DenseVector([0.0325, -0.1888])),
 Row(pcaFeatures=DenseVector([0.0266, -0.2294])),
 Row(pcaFeatures=DenseVector([0.0381, -0.1424])),
 Row(pcaFeatures=DenseVector([-0.4098, 0.8509])),
 Row(pcaFeatures=DenseVector([-0.4544, 0.8917])),
 Row(pcaFeatures=DenseVector([-0.4466, 0.8942])),
 Row(pcaFeatures=DenseVector([-0.4532, 0.9116])),
 Row(pcaFeatures=DenseVector([-0.1821, -0.0341])),
 Row(pcaFeatures=DenseVector([0.0362, -0.1625])),
 Row(pcaFeatures=DenseVector([0.0296, -0.1584])),
 Row(pcaFeatures=DenseVector([0.196, -0.0236])),
 Row(pcaFeatures=DenseVector([0.2732, -0.0943])),
 Row(pcaFeatures=DenseVector([-1.1855, -0.4257])),
 Row(pcaFeatures=DenseVector([-0.9809, -0.4748])),
 Row(pcaFeatures=DenseVector([-1.0788, -0.3982])),
 Row(pcaFeatures=DenseVector([-1.0289, -0.4125])),
 Row(pcaFeatures=DenseVector([0.0703, -0.1954])),
 Row(pcaFeatures=DenseVector([0.0188, -0.1911])),
 Row(pcaFeatures=DenseVector([-0.0149, -0.2162])),
 Row(pcaFeatures=DenseVector([0.0086, -0.2409])),
 Row(pcaFeatures=DenseVector([0.0397, 0.0148])),
 Row(pcaFeatures=DenseVector([-0.0048, 0.0824])),
 Row(pcaFeatures=DenseVector([0.0116, 0.0816])),
 Row(pcaFeatures=DenseVector([0.0531, 0.1271])),
 Row(pcaFeatures=DenseVecto

In [0]:
from pyspark.ml.functions import vector_to_array

df_pca_cluster = (model_df.withColumn("Axis", vector_to_array("pcaFeatures"))).select(["prediction"] + [col("Axis")[i] for i in range(2)])

In [0]:
df_pca_cluster.show()

+----------+--------------------+--------------------+
prediction| Axis[0]| Axis[1]|
+----------+--------------------+--------------------+
 0|-0.12451899344581047|-0.15504829355484703|
 0|-0.18095007880979413| -0.1437260393560732|
 3|-0.01902818142136...|0.023140156226589055|
 3| 0.06217762195685747|-0.06037605452855333|
 3| 0.11048161611147828| 0.02973177279337275|
 3|0.037669041237663806| 0.09502359189850186|
 3| 0.07253668883428993| 0.14231201459051324|
 2| 0.10633937164198622| -0.2897547660787983|
 2|-0.00960978270809...|-0.12626218069861042|
 2|-0.02623376925039...|-0.15861801288635982|
 3| 0.20249833270980266|-0.06366389260287902|
 3| 0.18559195353390692|-0.00906714423657...|
 3| 0.11497111036416155| 0.08680457040215993|
 3| 0.16795183270196257|0.046750731029054145|
 8| 0.25578267762207835|-0.34182512368748774|
 8| 0.24977604125037162|-0.21316366056781885|
 8| 0.26281955585943656|-0.24992310197712586|
 8| 0.22378493725862766|-0.21376056047161768|
 8| 0.29016883044286396| -0.2728291223759664|
 8| 0.29477318168811606| -0.3344542500774063|
+----------+--------------------+--------------------+
only showing top 20 rows

In [0]:
df_pca_cluster.createOrReplaceTempView('pcs_cluster_table')

In [0]:
%sql
select * from pcs_cluster_table

prediction,Axis[0],Axis[1]
0,-0.12451899344581047,-0.15504829355484703
0,-0.18095007880979413,-0.1437260393560732
3,-0.019028181421362277,0.023140156226589055
3,0.06217762195685747,-0.06037605452855333
3,0.11048161611147828,0.02973177279337275
3,0.037669041237663806,0.09502359189850186
3,0.07253668883428993,0.14231201459051324
2,0.10633937164198622,-0.2897547660787983
2,-0.009609782708093399,-0.12626218069861042
2,-0.026233769250398366,-0.15861801288635982


In [0]:
transformed.columns

Out[22]: ['ticker',
 'Sector',
 'Industry',
 'regularMarketOpen',
 'regularMarketDayHigh',
 'regularMarketDayLow',
 'regularMarketPreviousClose',
 'regularMarketVolume',
 'averageVolume',
 'averageDailyVolume10Day',
 'fiftyDayAverage',
 'marketCap',
 'Volume',
 'payoutRatio_new',
 'earningsQuarterlyGrowth_new',
 'fiveYearAvgDividendYield_new',
 'trailingAnnualDividendRate_new',
 'trailingPE_new',
 'dividendRate_new',
 'dividendYield_new',
 'socialScore_new',
 'governanceScore_new',
 'environmentScore_new',
 'fullTimeEmployees_new',
 'previousClose_new',
 'beta_new',
 'pegRatio_new',
 'enterpriseToRevenue_new',
 'sharesOutstanding_new',
 'bookValue_new',
 'heldPercentInsiders_new',
 'floatShares_new',
 'shortPercentOfFloat_new',
 'sharesShortPriorMonth_new',
 'profitMargins_new',
 'trailingEps_new',
 'shortRatio_new',
 'forwardPE_new',
 'previousClose',
 'enterpriseToEbitda_new',
 'Sector_Index',
 'Sector_classVec',
 'regularMarketOpen_vector',
 'regularMarketOpen_scaled',
 'regularMarketDayHigh_vector',
 'regularMarketDayHigh_scaled',
 'regularMarketDayLow_vector',
 'regularMarketDayLow_scaled',
 'regularMarketPreviousClose_vector',
 'regularMarketPreviousClose_scaled',
 'regularMarketVolume_vector',
 'regularMarketVolume_scaled',
 'averageVolume_vector',
 'averageVolume_scaled',
 'averageDailyVolume10Day_vector',
 'averageDailyVolume10Day_scaled',
 'fiftyDayAverage_vector',
 'fiftyDayAverage_scaled',
 'marketCap_vector',
 'marketCap_scaled',
 'Volume_vector',
 'Volume_scaled',
 'payoutRatio_new_vector',
 'payoutRatio_new_scaled',
 'earningsQuarterlyGrowth_new_vector',
 'earningsQuarterlyGrowth_new_scaled',
 'fiveYearAvgDividendYield_new_vector',
 'fiveYearAvgDividendYield_new_scaled',
 'trailingAnnualDividendRate_new_vector',
 'trailingAnnualDividendRate_new_scaled',
 'trailingPE_new_vector',
 'trailingPE_new_scaled',
 'dividendRate_new_vector',
 'dividendRate_new_scaled',
 'dividendYield_new_vector',
 'dividendYield_new_scaled',
 'socialScore_new_vector',
 'socialScore_new_scaled',
 'governanceScore_new_vector',
 'governanceScore_new_scaled',
 'environmentScore_new_vector',
 'environmentScore_new_scaled',
 'fullTimeEmployees_new_vector',
 'fullTimeEmployees_new_scaled',
 'previousClose_new_vector',
 'previousClose_new_scaled',
 'beta_new_vector',
 'beta_new_scaled',
 'pegRatio_new_vector',
 'pegRatio_new_scaled',
 'enterpriseToRevenue_new_vector',
 'enterpriseToRevenue_new_scaled',
 'sharesOutstanding_new_vector',
 'sharesOutstanding_new_scaled',
 'bookValue_new_vector',
 'bookValue_new_scaled',
 'heldPercentInsiders_new_vector',
 'heldPercentInsiders_new_scaled',
 'floatShares_new_vector',
 'floatShares_new_scaled',
 'shortPercentOfFloat_new_vector',
 'shortPercentOfFloat_new_scaled',
 'sharesShortPriorMonth_new_vector',
 'sharesShortPriorMonth_new_scaled',
 'profitMargins_new_vector',
 'profitMargins_new_scaled',
 'trailingEps_new_vector',
 'trailingEps_new_scaled',
 'shortRatio_new_vector',
 'shortRatio_new_scaled',
 'forwardPE_new_vector',
 'forwardPE_new_scaled',
 'previousClose_vector',
 'previousClose_scaled',
 'enterpriseToEbitda_new_vector',
 'enterpriseToEbitda_new_scaled',
 'features',
 'prediction']

In [0]:
# average of key variables impacting companies stocks

transformed.groupBy('prediction').avg('','marketCap','fiftyDayAverage','pegRatio_new','enterpriseToRevenue_new','profitMargins_new').orderBy('prediction').show()


+----------+--------------------+--------------------+-------------------+----------------------------+----------------------+
prediction| avg(marketCap)|avg(fiftyDayAverage)| avg(pegRatio_new)|avg(enterpriseToRevenue_new)|avg(profitMargins_new)|
+----------+--------------------+--------------------+-------------------+----------------------------+----------------------+
 0|1.085435852291282E11| 114.2024702051282| 13.132564102564102| 3.6471794871794883| -0.0978312783846154|
 1|5.698984211403174...| 108.58210038095235| 1.8047619047619048| 3.726317460317461| 0.1936871428888889|
 2| 2.8185825792E10| 143.2576822222222| 2.936111111111112| 3.5652777777777778| 0.06327388833333333|
 3|3.839874383842623E10| 141.78591509836068|-0.6177049180327869| 4.00967213114754| 0.08939819737704915|
 4|1.181873697621333...| 153.79662055000006| 2.545833333333333| 6.9460500000000005| 0.18432116551666666|
 5|6.918175954526315E10| 204.3303237192983| 5.435964912280701| 5.364438596491228| 0.15037736707017543|
 6|2.611491026971428...| 118.90237803571429| -0.899285714285714| 12.486500000000003| 0.20436071646428575|
 7|3.215405632474074E10| 72.63706559259259| 2.587037037037037| 5.088296296296296| 0.13366296425925925|
 8|5.993492977173333E10| 282.4952322666667| 3.9594999999999976| 3.657766666666667| -0.00961883371666...|
 9|7.494630282472728E10| 101.13045175757578| 3.327878787878788| 3.2064242424242413| 0.0913839396969697|
+----------+--------------------+--------------------+-------------------+----------------------------+----------------------+

In [0]:
transformed.select('ticker').show()

+------+
ticker|
+------+
 A|
 AAL|
 AAP|
 AAPL|
 ABBV|
 ABC|
 ABT|
 ACN|
 ADBE|
 ADI|
 ADM|
 ADP|
 ADSK|
 AEE|
 AEP|
 AES|
 AFL|
 AIG|
 AIZ|
 AJG|
+------+
only showing top 20 rows

In [0]:
# tickers for particular cluster
transformed.select('prediction', 'ticker','industry','sector').filter(col('prediction') == 8).show()

+----------+------+--------------------+-----------------+
prediction|ticker| industry| sector|
+----------+------+--------------------+-----------------+
 8| HBI|Apparel Manufactu...|Consumer Cyclical|
 8| PVH|Apparel Manufactu...|Consumer Cyclical|
 8| RL|Apparel Manufactu...|Consumer Cyclical|
 8| UAA|Apparel Manufactu...|Consumer Cyclical|
 8| VFC|Apparel Manufactu...|Consumer Cyclical|
 8| GPS| Apparel Retail|Consumer Cyclical|
 8| LB| Apparel Retail|Consumer Cyclical|
 8| ROST| Apparel Retail|Consumer Cyclical|
 8| TJX| Apparel Retail|Consumer Cyclical|
 8| KMX|Auto & Truck Deal...|Consumer Cyclical|
 8| F| Auto Manufacturers|Consumer Cyclical|
 8| GM| Auto Manufacturers|Consumer Cyclical|
 8| APTV| Auto Parts|Consumer Cyclical|
 8| BWA| Auto Parts|Consumer Cyclical|
 8| LKQ| Auto Parts|Consumer Cyclical|
 8| KSS| Department Stores|Consumer Cyclical|
 8| NKE|Footwear & Access...|Consumer Cyclical|
 8| FBHS|Furnishings, Fixt...|Consumer Cyclical|
 8| LEG|Furnishings, Fixt...|Consumer Cyclical|
 8| MHK|Furnishings, Fixt...|Consumer Cyclical|
+----------+------+--------------------+-----------------+
only showing top 20 rows

In [0]:
transformed.createOrReplaceTempView("stocks_clusters_data")


In [0]:
%sql
select  prediction as clusters, count(*) as number_of_companies from stocks_clusters_data group by clusters order by clusters

clusters,number_of_companies
0,39
1,63
2,18
3,61
4,60
5,57
6,28
7,27
8,60
9,33
